# Example RF Analysis

This script builds a very simple RandomForestRegressor from the sci-kitlearn library and shows how to do some basic analysis on it.


### Building the model

In [ ]:
import pandas as pd
import numpy as np
import xarray as xr

Read in the data from a specified section of years, and average it to a yearly value.

In [ ]:
startyear = 1979
endyear = 1995

folder = '~/Desktop/IMAS/'  #filepath
pfile = 'Proxy_combined_data_v4.nc'  #proxy filename
df = xr.open_dataset(folder+pfile).sel(year= slice(startyear, endyear))
df = df.mean('lon').mean('month')
df = df.to_dataframe()
df.reset_index(inplace=True)

Get the mean SIE, and partition the data into dependent and predictor variables.

In [ ]:
mean_sie = df['SIE'].mean()

X = df.iloc[:, 2:8].values
y = df.iloc[:, 1].values

Split the data into train and test sets, and then preprocess it with a standard scaler.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,
                                                    random_state = 0)

#Standardise values
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Train the model and generate a prediction.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 50, random_state = 0)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

---
### Analysing the model

Generate some metrics on error.

In [ ]:
from sklearn import metrics

rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', rmse)
print('RMSE % of mean:', 100*rmse/mean_sie)

Plot a tree from the model.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

fig = plt.figure(figsize = (15,10))
plot_tree(regressor.estimators_[0],
          feature_names = df.iloc[:, 2:8].columns,
          class_names = df['SIE'].unique(),
          filled = True, rounded = True)

plt.show()

Rank the importances of the proxies in the model.

In [ ]:
importances = pd.Series(regressor.feature_importances_, index=df.iloc[:, 2:8].columns)
importances = importances.sort_values(ascending=False)
print(importances)